In [3]:
from copy import deepcopy
import numpy as np

class Connect3Environment():
    def __init__(self, n_rows, n_cols):
        self.n_cols = n_cols
        self.n_rows = n_rows
        self.current_state = self.initialize_board()
        
    def initialize_board(self):
        return np.zeros((self.n_rows, self.n_cols))
    
    def initialize_board(self):
        return np.zeros((self.n_rows, self.n_cols))
    
    def reset(self):
        self.current_state = self.initialize_board()
        return self.current_state

    def is_terminal(self, state):
        reshaped_state = np.array(state).reshape(self.n_rows, self.n_cols)
        for row in range(4):
            for col in range(5):
                if reshaped_state[row][col] == 0:
                    continue
                player = reshaped_state[row][col]
                if (col <= 2 and all(reshaped_state[row][col+i] == player for i in range(3))) or \
                   (row <= 1 and all(reshaped_state[row+i][col] == player for i in range(3))) or \
                   (row <= 1 and col <= 2 and all(reshaped_state[row+i][col+i] == player for i in range(3))) or \
                   (row >= 2 and col <= 2 and all(reshaped_state[row-i][col+i] == player for i in range(3))):
                    return True
        # checking for draw also
        return not any(0 in row for row in reshaped_state)
    
    def get_possible_actions(self, state):
        reshaped_state = np.array(state).reshape(self.n_rows, self.n_cols)
        return [col for col in range(5) if reshaped_state[0, col] == 0]
    
    def step(self, action):
        #prev_step = deepcopy(self.current_state)
        reshaped_board = np.array(self.current_state).reshape(self.n_rows, self.n_cols)
        # ruch agenta
        for row in range(3, -1, -1):
            if reshaped_board[row, action]== 0:
                reshaped_board[row, action] = 1
                break
        if self.is_terminal(reshaped_board):
            return reshaped_board, 1, True
        
        # ruch przeciwnika
        opponents_action = np.random.choice(self.get_possible_actions(reshaped_board))
        for row in range(3, -1, -1):
            if reshaped_board[row, opponents_action]== 0:
                reshaped_board[row, opponents_action] = -1
                break
        if self.is_terminal(reshaped_board):
            return reshaped_board, -1, True
        self.current_state = reshaped_board
        
        return self.current_state, 0, self.is_terminal(self.current_state)

In [ ]:
import random

class FunctionApproximationAgent:
    def __init__(self, alpha, gamma, epsilon, get_legal_actions):
        self.cache = {}
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.theta_action0 = np.zeros((41, 1))
        self.theta_action1 = np.zeros((41, 1))
        self.theta_action2 = np.zeros((41, 1))
        self.theta_action3 = np.zeros((41, 1))
        self.theta_action4 = np.zeros((41, 1))
        
        self.theta_dict = {0: np.array([[ 0.00000000e+00],
                        [-3.95269174e-02],
                        [-3.45740362e-01],
                        [-5.77726982e-02],
                        [-2.74688350e-02],
                        [ 1.24363031e-02],
                        [ 2.62351874e-02],
                        [-1.73750168e-03],
                        [-6.84586759e-02],
                        [-3.99119556e-02],
                        [-6.71827303e-02],
                        [-9.92370755e-02],
                        [ 2.32820568e-02],
                        [-1.25252460e-01],
                        [-4.04794802e-01],
                        [-4.90317472e-01],
                        [ 3.05447132e-01],
                        [ 1.05965089e+00],
                        [ 5.26691409e-01],
                        [-1.77935730e-01],
                        [ 0.00000000e+00],
                        [ 2.67123754e-02],
                        [ 2.58970062e-01],
                        [-7.45922525e-03],
                        [-1.50867587e-02],
                        [-2.19267512e-01],
                        [ 1.51322412e-02],
                        [-3.14950872e-01],
                        [-1.16391331e-02],
                        [-4.53650483e-02],
                        [ 6.39783990e-01],
                        [ 4.32725371e-01],
                        [ 1.46990660e-01],
                        [ 1.93574398e-01],
                        [ 3.44024981e-01],
                        [ 1.79210975e-01],
                        [-1.90117141e-01],
                        [-1.43894590e+00],
                        [-3.28100543e-01],
                        [ 3.42212846e-01],
                        [ 4.41480256e+00]]),
                1: np.array([[-1.45237206e-02],
                        [ 0.00000000e+00],
                        [-3.73705884e-01],
                        [-4.22832247e-02],
                        [-5.17735513e-02],
                        [ 5.29805635e-02],
                        [-6.41267765e-02],
                        [ 4.60926486e-03],
                        [-3.07326500e-02],
                        [-4.34477647e-02],
                        [-4.72613015e-01],
                        [ 5.31762889e-02],
                        [ 2.05336233e-01],
                        [-3.35906872e-04],
                        [-4.52579717e-01],
                        [ 2.00437281e-02],
                        [-3.36099381e-01],
                        [ 1.12491925e+00],
                        [ 6.47338979e-01],
                        [-2.07113189e-01],
                        [ 7.39635409e-02],
                        [ 0.00000000e+00],
                        [ 4.56718883e-01],
                        [ 7.50255880e-02],
                        [ 2.29564040e-02],
                        [ 6.91039058e-02],
                        [-2.11410320e-01],
                        [-2.01370987e-01],
                        [ 1.33807951e-01],
                        [ 7.49297455e-02],
                        [ 5.23371784e-01],
                        [ 6.01722122e-01],
                        [ 2.33791635e-01],
                        [ 2.49398212e-01],
                        [ 5.86961214e-01],
                        [ 6.26027808e-01],
                        [-9.71373600e-01],
                        [-1.79712032e+00],
                        [-8.70562998e-01],
                        [ 3.43128954e-01],
                        [ 4.93625072e+00]]),
                2: np.array([[-6.24784059e-02],
                        [-2.41954953e-03],
                        [ 0.00000000e+00],
                        [-5.59783097e-02],
                        [-2.78115604e-02],
                        [-8.17173446e-02],
                        [ 3.85448141e-01],
                        [-4.40272999e-01],
                        [ 1.38824756e-01],
                        [-5.01629251e-02],
                        [-5.22411008e-01],
                        [-2.91567639e-01],
                        [ 4.19982587e-01],
                        [-1.52794286e-02],
                        [-4.73698551e-01],
                        [-2.58930121e-01],
                        [ 1.27921846e-01],
                        [ 2.58601224e-02],
                        [ 4.91160394e-01],
                        [-1.26080678e-02],
                        [ 4.45503731e-02],
                        [ 1.71379997e-01],
                        [ 0.00000000e+00],
                        [ 1.12812431e-01],
                        [ 6.30188190e-02],
                        [ 6.21825153e-02],
                        [ 2.21096288e-01],
                        [-1.30977007e+00],
                        [ 9.12354589e-02],
                        [ 1.24932903e-01],
                        [ 4.73632284e-01],
                        [ 4.77188227e-01],
                        [ 1.03767893e-01],
                        [ 2.13435029e-01],
                        [ 5.71781044e-01],
                        [ 6.02978444e-01],
                        [-4.15733215e-02],
                        [-2.48217184e+00],
                        [-5.79472675e-01],
                        [ 3.72858139e-01],
                        [ 5.62368936e+00]]),
                3: np.array([[-6.49564875e-02],
                        [-1.08356532e-03],
                        [-4.46317331e-01],
                        [ 0.00000000e+00],
                        [-3.14198192e-02],
                        [-8.85423195e-02],
                        [ 2.33295460e-03],
                        [ 8.09987737e-02],
                        [-1.64309945e-01],
                        [ 2.61255442e-01],
                        [-5.31287730e-01],
                        [-2.76053897e-01],
                        [ 1.22154534e-01],
                        [ 2.10557590e-01],
                        [-4.06652169e-01],
                        [-4.49240787e-01],
                        [ 3.27959882e-01],
                        [ 1.02558312e+00],
                        [-8.84815715e-02],
                        [ 2.27451772e-01],
                        [ 3.23902739e-02],
                        [ 8.20308438e-02],
                        [ 4.92691444e-01],
                        [ 0.00000000e+00],
                        [ 1.42377128e-01],
                        [ 3.45655960e-02],
                        [ 2.60984982e-01],
                        [-2.62493349e-01],
                        [-5.73541736e-01],
                        [ 1.24845111e-01],
                        [ 4.59433098e-01],
                        [ 4.40663516e-01],
                        [ 1.79079384e-01],
                        [ 9.50808676e-02],
                        [ 6.23552361e-01],
                        [ 6.14419800e-01],
                        [-4.11595842e-02],
                        [-1.74537529e+00],
                        [-1.56885636e+00],
                        [ 3.19260357e-01],
                        [ 5.21878661e+00]]),
                4: np.array([[-3.86883965e-02],
                        [-3.57641926e-02],
                        [-2.86149981e-01],
                        [-9.70152443e-02],
                        [ 0.00000000e+00],
                        [-4.92355476e-02],
                        [-5.34165746e-02],
                        [ 4.07473162e-04],
                        [-6.57024421e-03],
                        [ 6.77447819e-02],
                        [-4.28785500e-01],
                        [-2.82313297e-01],
                        [ 5.47994685e-02],
                        [ 1.12429513e-01],
                        [-1.25538173e-01],
                        [-3.91086255e-01],
                        [ 1.51564442e-03],
                        [ 1.07780825e+00],
                        [ 7.48641140e-01],
                        [-3.85928221e-01],
                        [-9.56058783e-03],
                        [ 7.99245201e-04],
                        [ 3.26237685e-01],
                        [ 5.64584203e-02],
                        [ 0.00000000e+00],
                        [-3.24312198e-02],
                        [ 8.63501144e-02],
                        [-2.89008887e-01],
                        [ 4.54905005e-02],
                        [-1.36198861e-01],
                        [ 4.34517278e-01],
                        [ 4.35181026e-01],
                        [ 2.01813672e-01],
                        [ 1.68150994e-01],
                        [ 7.06857870e-01],
                        [ 6.18631902e-01],
                        [ 9.79101052e-02],
                        [-1.66538843e+00],
                        [-7.49131573e-01],
                        [-4.13824605e-01],
                        [ 4.71202250e+00]])}
        
        self.get_legal_actions = get_legal_actions
    
    def generate_feature_vector(self, board):
        arr1 = board==1
        arr2 = board==-1
        return [*arr1.flatten()]+[*arr2.flatten()]+[1] 

    def calculate_value(self, state, action):
        feature_vector = self.generate_feature_vector(state)
        value = feature_vector@self.theta_dict[action]
        return value
    
    def get_value(self, state):
        possible_actions = self.get_legal_actions(state)
        
        if len(possible_actions) == 0:
            return 0.0
        
        max_value = float('-inf')
        for action in possible_actions:
            feature_vector_action = self.calculate_value(state, action)
            if max_value < feature_vector_action:
                max_value = feature_vector_action
        return max_value
    
    def update(self, state, action, reward, next_state):
        valid_actions_next_state = self.get_legal_actions(next_state)
        if len(valid_actions_next_state) > 0:
            value_next_state = self.get_value(next_state)
        else:
            value_next_state = 0
        error = reward+self.gamma*value_next_state-self.calculate_value(state,action)
        delta_theta = self.alpha*error*self.generate_feature_vector(state)
        self.theta_dict[action] = self.theta_dict[action]+delta_theta.reshape(self.theta_action1.shape)
        
    def get_best_action(self, state):
        possible_actions = self.get_legal_actions(state)
        
        if len(possible_actions) == 0:
            return None
        
        best_actions = []
        action_value = float('-inf')
        for action in possible_actions:
            value = self.calculate_value(state, action)
            if action_value < value:
                best_actions = [action]
                action_value = value
            elif action_value == value:
                best_actions.append(action)
        best_action = np.random.choice(best_actions)
        return best_action
    
    def get_action(self, state):
        # Pick Action
        possible_actions = self.get_legal_actions(state)

        # If there are no legal actions, return None
        if len(possible_actions) == 0:
            return None

        random_number = random.random()
        if random_number <= self.epsilon:
            exploration = True
        else:
            exploration = False
        
        if exploration == False:
            chosen_action = self.get_best_action(state)
        else:
            if len(possible_actions)>1:
                # possible_actions.remove(self.get_best_action(state))
                chosen_action = random.choice(possible_actions)
            else: 
                chosen_action = possible_actions[0]
        
        return chosen_action
        
    def turn_off_learning(self):
        """
        Function turns off agent learning.
        """
        self.epsilon = 0
        self.alpha = 0

In [8]:
def play_and_train(env, agent):
    """
    This function should
    - run a full game, actions given by agent's e-greedy policy
    - train agent using agent.update(...) whenever it is possible
    - return total reward
    """
    total_reward = 0.0
    state = env.reset()
    done = False
    
    while not done:
        # get agent to pick action given state state.
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)
        #
        # INSERT CODE HERE to train (update) agent for state
        #        
        agent.update(state, action, reward, next_state)

        state = next_state
        total_reward += reward
        if done:
            break
    return total_reward

In [197]:
environment = Connect3Environment(4, 5)
agent = FunctionApproximationAgent(0.0001, 0.9, 0.15, environment.get_possible_actions)

In [198]:
for i in range(500000):
    play_and_train(environment, agent)

In [203]:
agent.turn_off_learning()
play_and_train(environment, agent)

1.0

In [206]:
import os

def game_is_terminal(state):
    reshaped_state = np.array(state).reshape(4,5)
    #print(reshaped_state)
    for row in range(4):
        for col in range(5):
            if reshaped_state[row][col] == 0:
                continue
            player = reshaped_state[row][col]
            if (col <= 2 and all(reshaped_state[row][col+i] == player for i in range(3))) or \
               (row <= 1 and all(reshaped_state[row+i][col] == player for i in range(3))) or \
               (row <= 1 and col <= 2 and all(reshaped_state[row+i][col+i] == player for i in range(3))) or \
               (row >= 2 and col <= 2 and all(reshaped_state[row-i][col+i] == player for i in range(3))):
                return True
    # checking for draw also
    return not any(0 in row for row in reshaped_state)

def game_step(board, action, player):
    next_state = deepcopy(board)
    for row in range(3, -1, -1):
        if next_state[row, action] == 0:
            next_state[row, action] = player
            break
    
    return next_state, game_is_terminal(next_state)

def game(env, agent):
    clear = lambda: os.system('cls')
    
    player_round = 1
    board = env.reset()
    print(board)
    
    while True:
        if player_round == 1:
            agent_move = agent.get_action(board)
            board, done = game_step(board, agent_move, player_round)
            player_round = 0 - player_round
        else:
            player_move = int(input('Podaj swój ruch (0-4): '))
            board, done = game_step(board, player_move, player_round)
            player_round = 0 - player_round
        #clear()
        print(board)
        
        if env.is_terminal(board):
            print(f'Wygrał gracz {0 - player_round}')
            break

In [217]:
agent.turn_off_learning()
game(environment, agent)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0. -1.]]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1. -1.]]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0. -1.  1.  1. -1.]]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0. -1.  1.  1. -1.]]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0. -1.  1.  1. -1.]]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  1.  1.  0.]
 [ 0. -1.  1.  1. -1.]]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  1.  1. -1.]
 [ 0. -1.  1.  1. -1.]]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0. -1.  1.  0.]
 [ 0.  0.  1.  1. -1.]
 [ 0. -1.  1.  1. -1.]]
Wygrał gracz 1
